<a href="https://colab.research.google.com/gist/ShayGali/9f7a94ecfccf90047e5c1662c050bdc5/untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Economic Algorithms Assignment 11

In [1]:
import networkx as nx
from typing import List, Set, Optional


def find_decomposition(
    budget: List[float],
    preferences: List[Set[int]],
) -> Optional[List[List[float]]]:
    """
    Check whether a given continuous budget vector is decomposable, and
    return one valid decomposition if it exists.

    Parameters
    ----------
    budget : list[float]
        The amount d_j allocated to each project j.  The sum equals C.
    preferences : list[set[int]]
        For every citizen i, the set A_i of projects she approves.

    Returns
    -------
    None | list[list[float]]
        * None              - the budget is **not** decomposable.
        * list of lists     - an n X m matrix d_{i,j}; each row i sums
          to C/n, each column j sums to d_j, and d_{i,j} > 0 only if
          j ∈ A_i.
    """

    n = len(preferences)
    m = len(budget)
    C = sum(budget)

    # Each citizen’s “fair share” according to the assignment text
    share = C / n

    # Build a flow network:
    #   source  ->  citizens  ->  projects  ->  sink
    G = nx.DiGraph()
    src, sink = "s", "t"

    # Source → citizen edges: capacity
    for i in range(n):
        G.add_edge(src, f"i{i}", capacity=share)

    # Citizen → project edges: capacity = share
    for i, proj in enumerate(preferences):
        for j in proj:
            G.add_edge(f"i{i}", f"p{j}", capacity=share)

    # Project → sink edges: capacity = budget assigned to the project
    for j, dj in enumerate(budget):
        G.add_edge(f"p{j}", sink, capacity=dj)

    # Compute the maximum flow
    flow_value, flow_dict = nx.maximum_flow(G, src, sink)

    # If the max flow is smaller than C (up to numerical tolerance),
    # the budget cannot be decomposed.
    if abs(flow_value - C) > 1e-6:
        return None

    # Extract the decomposition matrix d_{i,j} from the flow result
    decomposition = [
        [flow_dict.get(f"i{i}").get(f"p{j}", 0.0) for j in range(m)]  # 0 if no edge
        for i in range(n)
    ]

    return decomposition


# ---- Example usage ----
if __name__ == "__main__":
    budget = [400, 50, 50, 0]  # d_j (must sum to C = 500)
    preferences = [{0, 1}, {0, 2}, {0, 3}, {1, 2}, {0}]  # A_i sets

    d = find_decomposition(budget, preferences)
    if d is None:
        print("The budget is NOT decomposable.")
    else:
        print("A valid decomposition matrix d_{i,j}:")
        for i, row in enumerate(d):
            print(f"Citizen {i}: {row}")


A valid decomposition matrix d_{i,j}:
Citizen 0: [100.0, 0, 0.0, 0.0]
Citizen 1: [100.0, 0.0, 0, 0.0]
Citizen 2: [100.0, 0.0, 0.0, 0]
Citizen 3: [0.0, 50.0, 50.0, 0.0]
Citizen 4: [100.0, 0.0, 0.0, 0.0]
